In [1]:
from google.colab import drive
drive.mount('/content/drive')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pickle
import os
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm

# # Define the subject list, subsets, and tasks
subject_list = (1, 2, 3, 4, 5, 6, 7, 8)
subsets = ['train', 'val']
tasks = ['normal', 'depth', 'general_pose', 'seg']
rois = ['FBA_2','V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'EBA', 'OFA', 'FFA_1', 'FFA_2', 'FBA_1']



Mounted at /content/drive


In [2]:
def extract_rdms(nn_rdms, brain_rdms, subject_int, roi, hemi='concatenated'):
  tasks = ['general_pose', 'seg', 'depth', 'normal']
  subj = subject_int  # use int if keys in nn_rdms are int
  nn_rdms_train = {task: nn_rdms[str(subj)][task]['train'][roi][hemi] for task in tasks}
  nn_rdms_val = {task: nn_rdms[str(subj)][task]['val'][roi][hemi] for task in tasks}
  brain_rdm_train = brain_rdms['train'][subj][roi][hemi]
  brain_rdm_val = brain_rdms['val'][subj][roi][hemi]
  return nn_rdms_train, nn_rdms_val, brain_rdm_train, brain_rdm_val

def get_model_r2_linear(X_train, y_train, X_val, y_val):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return r2_score(y_val, y_pred)

In [3]:
# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Imports ---
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# --- Subject and model info ---
subject_list = (1, 2, 3, 4, 5, 6, 7, 8)
tasks = ['normal', 'depth', 'general_pose', 'seg']
rois = ['FBA_2', 'V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'EBA', 'OFA', 'FFA_1', 'FFA_2', 'FBA_1']
model_types = ['head', 'transformer']

# --- Paths ---
rdm_paths = {
    'head': '/content/drive/MyDrive/ML Project/RDM_results/best_rdms_final_sapiens_head.pkl',
    'transformer': '/content/drive/MyDrive/ML Project/RDM_results/best_rdms_final_sapiens_transformer.pkl'
}
PATH_TO_BRAIN_RDMs = '/content/drive/MyDrive/rdm_brain_rois_2.pkl'

# --- Functions ---
def extract_rdms(nn_rdms, brain_rdms, subject_int, roi, hemi='concatenated'):
    tasks = ['general_pose', 'seg', 'depth', 'normal']
    subj = subject_int
    nn_rdms_train = {task: nn_rdms[str(subj)][task]['train'][roi][hemi] for task in tasks}
    nn_rdms_val = {task: nn_rdms[str(subj)][task]['val'][roi][hemi] for task in tasks}
    brain_rdm_train = brain_rdms['train'][subj][roi][hemi]
    brain_rdm_val = brain_rdms['val'][subj][roi][hemi]
    return nn_rdms_train, nn_rdms_val, brain_rdm_train, brain_rdm_val

def get_model_r2_linear(X_train, y_train, X_val, y_val):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    r2 = r2_score(y_val, y_pred)
    betas = model.coef_
    intercept = model.intercept_
    return r2, betas, intercept

# --- Load RDMs ---
nn_rdms_all = {}
for model_type in model_types:
    with open(rdm_paths[model_type], 'rb') as f:
        nn_rdms_all[model_type] = pickle.load(f)

with open(PATH_TO_BRAIN_RDMs, 'rb') as file:
    brain_rdms = pickle.load(file)

# --- Precompute keys ---
tasks_sorted = sorted(tasks)
full_model_key = '+'.join(tasks_sorted)
leave_one_out_keys = {
    task: '+'.join(t for t in tasks_sorted if t != task)
    for task in tasks_sorted
}
combo_keys = [full_model_key] + list(leave_one_out_keys.values())

# --- Storage dictionaries ---
roi_subjectwise_data = {
    model_type: {
        roi: {
            **{f'unique_{task}': [] for task in tasks_sorted},
            **{f'single_{task}_r2': [] for task in tasks_sorted},
            **{f'single_{task}_betas': [] for task in tasks_sorted},
            **{f'single_{task}_intercepts': [] for task in tasks_sorted},
            'shared': [],
            'full_model_r2': [],
            'full_model_betas': [],
            'full_model_intercepts': []
        }
        for roi in rois
    }
    for model_type in model_types
}

roi_results = {
    model_type: {}
    for model_type in model_types
}

# --- Main Loop ---
for model_type in model_types:
    nn_rdms = nn_rdms_all[model_type]
    for roi in rois:
        for subj in subject_list:
            try:
                nn_tr, nn_val, br_tr, br_val = extract_rdms(nn_rdms, brain_rdms, subj, roi)

                r2_scores = {}
                betas_store = {}

                # --- Fit full model and leave-one-out models ---
                for combo_key in combo_keys:
                    combo = combo_key.split('+')
                    X_tr = np.stack([nn_tr[t] for t in combo], axis=1)
                    X_val = np.stack([nn_val[t] for t in combo], axis=1)

                    r2, betas, intercept = get_model_r2_linear(X_tr, br_tr, X_val, br_val)

                    r2_scores[combo_key] = r2
                    betas_store[combo_key] = (betas, intercept)

                full_r2 = r2_scores[full_model_key]

                # --- Compute uniques ---
                uniques = {
                    f'unique_{task}': full_r2 - r2_scores[leave_one_out_keys[task]]
                    for task in tasks_sorted
                }

                shared = full_r2 - sum(uniques.values())

                # --- Fit each single model alone ---
                for task in tasks_sorted:
                    X_tr_single = nn_tr[task][:, None]  # (n_samples, 1)
                    X_val_single = nn_val[task][:, None]

                    r2_single, betas_single, intercept_single = get_model_r2_linear(X_tr_single, br_tr, X_val_single, br_val)

                    roi_subjectwise_data[model_type][roi][f'single_{task}_r2'].append(r2_single)
                    roi_subjectwise_data[model_type][roi][f'single_{task}_betas'].append(betas_single)
                    roi_subjectwise_data[model_type][roi][f'single_{task}_intercepts'].append(intercept_single)

                # --- Store per subject ---
                for name, val in uniques.items():
                    roi_subjectwise_data[model_type][roi][name].append(val)

                roi_subjectwise_data[model_type][roi]['shared'].append(shared)
                roi_subjectwise_data[model_type][roi]['full_model_r2'].append(full_r2)
                roi_subjectwise_data[model_type][roi]['full_model_betas'].append(betas_store[full_model_key][0])
                roi_subjectwise_data[model_type][roi]['full_model_intercepts'].append(betas_store[full_model_key][1])

            except Exception as e:
                print(f"Skipping subj={subj}, roi={roi}, model={model_type}: {e}")

        # --- Aggregate across subjects ---
        roi_results[model_type][roi] = {
            metric: np.mean(values, axis=0) if isinstance(values[0], (np.ndarray, list)) else np.mean(values)
            for metric, values in roi_subjectwise_data[model_type][roi].items()
        }


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Skipping subj=1, roi=FBA_2, model=head: 'FBA_2'
Skipping subj=1, roi=FFA_2, model=head: 'FFA_2'
Skipping subj=2, roi=FBA_1, model=head: 'FBA_1'
Skipping subj=4, roi=FBA_1, model=head: 'FBA_1'
Skipping subj=7, roi=FBA_1, model=head: 'FBA_1'
Skipping subj=1, roi=FBA_2, model=transformer: 'FBA_2'
Skipping subj=1, roi=FFA_2, model=transformer: 'FFA_2'
Skipping subj=2, roi=FBA_1, model=transformer: 'FBA_1'
Skipping subj=4, roi=FBA_1, model=transformer: 'FBA_1'
Skipping subj=7, roi=FBA_1, model=transformer: 'FBA_1'


In [4]:
roi_results

{'head': {'FBA_2': {'unique_depth': np.float64(0.0006426130233601216),
   'unique_general_pose': np.float64(-0.0022731855326172073),
   'unique_normal': np.float64(-0.0016333244484888962),
   'unique_seg': np.float64(0.016057698193157357),
   'single_depth_r2': np.float64(-0.004606893863679311),
   'single_general_pose_r2': np.float64(-0.00561608751711884),
   'single_normal_r2': np.float64(-0.007298536947171578),
   'single_seg_r2': np.float64(0.013519600269832581),
   'single_depth_betas': array([0.02919888]),
   'single_general_pose_betas': array([-0.07218957]),
   'single_normal_betas': array([0.02868097]),
   'single_seg_betas': array([-0.14217027]),
   'single_depth_intercepts': np.float64(0.6114535675418324),
   'single_general_pose_intercepts': np.float64(0.6114535675418324),
   'single_normal_intercepts': np.float64(0.6114535675418324),
   'single_seg_intercepts': np.float64(0.6114535675418324),
   'shared': np.float64(-0.0018283677284273),
   'full_model_r2': np.float64(0.010